# Numba example

In [4]:
from numba import njit, jit
import numba.cuda as cuda
import random


points = 100000

def pi(npoints):
    n_in_circle = 0
    for i in range(npoints):
        x = random.random()
        y = random.random()
        if (x**2+y**2 < 1):
            n_in_circle += 1
    return 4*n_in_circle / npoints

pi(points)

3.13928

In [2]:
import random
from numba import njit
@njit # or @jit(nopython=True)
def fast_pi(npoints):
    n_in_circle = 0
    for i in range(npoints):
        x = random.random()
        y = random.random()
        if (x**2+y**2 < 1):
            n_in_circle += 1
    return 4*n_in_circle / npoints

fast_pi(points)

3.1425048

%timeit permet de vérifier le temps que prend une fonction en effectuant plusieurs fois la fonction. Loop=nombre de fois que la fonction est lancée pour chaque run. Run=recommencement du test

In [5]:
%timeit pi(points)

101 ms ± 2.88 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit fast_pi(points)

2.19 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
# @cuda.jit
# def very_fast_pi(npoints):
#     n_in_circle = 0
#     for i in range(npoints):
#         x = random.random()
#         y = random.random()
#         if (x**2+y**2 < 1):
#             n_in_circle += 1
#     return 4*n_in_circle / npoints


Unfortunately a naive @cuda.jit decorator doesn't work. When you want to use this decorator, you have to write a kernel, which looks like C++ (but is not!). So, Numba is not a drop-in replcaement for GPU calculation, but an excellent CPU optimizer and I recommend using the @njit decorator each time you have a code that has
- loops
- matrice itteration
- matrice conditions
- numpy arrays
- 

# Cupy example

In [ ]:
import numpy as np

points = 1000000

def numpy_pi(npoints):
    count = 0
    xy = np.random.rand(2, npoints)
    for i in range(len(xy[0])):
        if (xy[0, i]**2 + xy[1, i]**2) < 1:
            count += 1
    return 4*count/npoints

numpy_pi(points)


In [ ]:
numpy_pi(points)

In [ ]:
import cupy as cp

points = 1000000

def cupy_pi(npoints):
    count = 0
    xy = cp.random.rand(2, npoints)
    for i in range(len(xy[0])):
        if (xy[0, i]**2 + xy[1, i]**2) < 1:
            count += 1
    return 4*count/npoints

cupy_pi(points)

In [ ]:
cupy_pi(points)

# Cupy in the real world

Here's a function that we use in PyTissueOptics for the photons diffusion simulator

In [32]:
import numpy as np
import cupy as cp

def randomUniformUnitary(N):
    theta = np.random.rand(N) * 2 * np.pi
    phi = np.random.rand(N) * np.pi
    x = np.sin(phi)*np.cos(theta)
    y = np.sin(phi)*np.sin(theta)
    z = np.cos(phi)
    output = np.stack((x, y, z), axis=-1)
#     print("done")

def randomUniformUnitaryCupy(N):
    theta = cp.random.rand(N) * 2 * cp.pi
    phi = cp.random.rand(N) * cp.pi
    x = cp.sin(phi)*cp.cos(theta)
    y = cp.sin(phi)*cp.sin(theta)
    z = cp.cos(phi)
    output = cp.stack((x, y, z), axis=-1)
#     print("done")
    
points = int(1e5)

In [33]:
randomUniformUnitary(points)

In [34]:
randomUniformUnitaryCupy(points)

In [42]:
import time
def batches_numpy():
    a = time.time_ns()
    batches = 1000
    for i in range(batches):
        randomUniformUnitary(points)
    b = time.time_ns()
    print("numpy {} batches of {}: {}ms total".format(batches, points, int(b-a)/1000000))

def batches_cupy():
    a = time.time_ns()
    batches = 100000
    for i in range(batches):
        randomUniformUnitaryCupy(points)
    b = time.time_ns()
    print("cupy {} batches of {}: {}ms total".format(batches, points, int(b-a)/1000000))

In [40]:
batches_numpy()

numpy 1000 batches of 100000: 12234.0015ms total


In [43]:
batches_cupy()

cupy 100000 batches of 100000: 47122.0052ms total
